In [2]:
import numpy as np
from scipy.io.matlab import loadmat
from scipy.optimize import linear_sum_assignment

# HW3

In this homework we are going to look at a hyperspectral image dataset called [Salinas](https://rslab.ut.ac.ir/data) dataset.

This scene was collected by the 224-band AVIRIS sensor over Salinas Valley, California, and is characterized by high spatial resolution (3.7-meter pixels). The area covered comprises 512´217. As with Indian Pines scene, we discarded the 20 water absorption bands, in this case bands: [108-112], [154-167], 224. This image was available only as at-sensor radiance data. It includes vegetables, bare soils, and vineyard fields. Salinas ground truth contains 16 classes.

In [3]:
salinas = loadmat('data/Salinas_corrected.mat')['salinas_corrected']
salinas.shape

(512, 217, 204)

In [5]:
salinas_gt = loadmat('data/Salinas_gt.mat')['salinas_gt']
salinas_gt.shape

(512, 217)

## Q1 

1. Reshape and split the data into train (80%) and test (20%) data sets.
2. Apply all of the supervised ML algorithms we learned on the data and compare the results.

***
### Logistic Regression

***
## Q2

Now, apply all of the unsupervised algorithms we have learned so far and compare the results. In order to align the unsupervised labels and true labels use the `linear_sum_assignment` function from `scipy.optimize`. For example, if a confusion matrix looks like:

In [30]:
cm = np.array(
      [[1,10,2],
       [9,2,1],
       [3,4,10]])

we get

In [32]:
rows, cols = linear_sum_assignment(cm,maximize=True)
cm = cm[rows,:]
cm = cm[:,cols]
cm

array([[10,  1,  2],
       [ 2,  9,  1],
       [ 4,  3, 10]])

## Q3

Now compare supervised and unsupervised algorithm results.